In [1]:
#import modules
import pandas as pd
import numpy as np
from datetime import datetime
#See max columns in a dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
# Read the CSV file with specified data types and skipped first 4 rows
vacant_lots_df = pd.read_csv("data/tolemi-export1692668115380.csv",skiprows=4)


In [3]:
#To see the rows before dropping duplicates
vacant_lots_df.shape

(25000, 26)

In [4]:
# Remove duplicates based on all columns
vacant_lots_df = vacant_lots_df.drop_duplicates(subset=['Parcel ID'])


In [5]:
#Rows after duplicates have been dropped
vacant_lots_df.shape

(24901, 26)

In [6]:
#Creates field Property Age Years in df

#Assign a variable to the current year
current_year = 2023

# Calculate "Property Age Years" as the difference between 2023 and Year Built
vacant_lots_df['Property Age Years'] = current_year - vacant_lots_df['Year Built']

# Assign the calculated columns to variables for potential future use
property_age_years = vacant_lots_df['Property Age Years']


In [7]:
#Creates field Vacancy Days in df

# Convert today's date to a Timestamp object
today = pd.Timestamp(datetime.today().date())

# Convert the 'Vacant Since' column to a Datetime column 
vacant_lots_df['Vacant Since'] = pd.to_datetime(vacant_lots_df['Vacant Since'])

# Calculate "Vacancy Days" as the difference between today's date and "Vacant Since" date
vacant_lots_df['Vacancy Days'] = (today - vacant_lots_df['Vacant Since']).dt.days

# Assign the calculated columns to variables for potential future use
vacancy_days = vacant_lots_df['Vacancy Days']

In [8]:

#For ending set of 3 questions use a function to map Single Family,Multi, and Commercial
def zoning_triple_category(row):
    zoning=row['Zoning']
    
    if pd.notna(zoning):#check if thed value is not NaN
        if "Single Family" in zoning or "Mobile Home" in zoning:
            return "Single Family"
        elif any(keyword in zoning for keyword in ["Multifamily", "Duplex", "Cluster Housing", "Townhouse"]):
            return "Multifamily"
        elif "Commercial" in zoning or "Commerical" in zoning:
            return "Commercial"
        return "Other" #Return "Other" for NaN or missing values
vacant_lots_df['Zoning Category 3'] = vacant_lots_df.apply(zoning_triple_category, axis=1)  

In [9]:
# Check if "CHURCH" is a partial match in the "Current Use" column and create the "Owner is Church" column
vacant_lots_df['Owner is Church']=vacant_lots_df['Current Use'].str.contains('CHURCH', case=False, na=False)

# Map the True/False values to 'Yes' and 'No' in the "Owner is Church" column
vacant_lots_df['Owner is Church'] = vacant_lots_df['Owner is Church'].map({True: 'Yes', False: 'No'})

owner_is_church=vacant_lots_df["Owner is Church"]

In [10]:

#Checks strings in zoning column and maps them
def classify_zoning_category(row):
    zoning = row['Zoning']
    
    if pd.notna(zoning):  # Check if the value is not NaN
        if "Central Area" in zoning:
            return "Central Area"
        elif "Retail" in zoning or "Neighborhood Service" in zoning:
            return "Retail"
        elif any(keyword in zoning for keyword in ["Single Family", "Multifamily", "Duplex", "Cluster Housing", "Townhouse"]):
            return "Residential"
        elif "Office" in zoning:
            return "Office"
        elif "Industrial" in zoning:
            return "Industrial"
        elif "Commercial" in zoning or "Commerical" in zoning:  # Handle the typo
            return "Commercial"
    return "Other"  # Return "Other" for NaN or missing values

# Apply the function to create the "Zoning Category" column
vacant_lots_df['Zoning Category'] = vacant_lots_df.apply(classify_zoning_category, axis=1)

In [11]:
# Define a function to apply the classification logic
def classify_owner(row):
    if pd.notna(row['Public Owner']):
        return row['Public Owner']
    elif row['Public Owner'] == 'City of Dallas':
        return 'City of Dallas'
    elif row['Public Owner'] == 'City of Dallas Housing Authority':
        return 'City of Dallas Housing Authority'
    elif row['Public Owner'] == 'County of Dallas':
        return 'County of Dallas'  
    elif row['Owner is Church'] == 'Yes':
        return 'Church'
    elif row['Owner Is Business'] == 'Yes':
        return 'Business'
    elif row['Bank Owned Property'] == 'Yes':
        return 'Bank'
    else:
        return 'Private Owner'

# Apply the function to create the "Owner" column
vacant_lots_df['Owner'] = vacant_lots_df.apply(classify_owner, axis=1)
   

In [12]:
# Selecting columns of interest and filtering by 'Property Type' == 'Structure'
structure_df = vacant_lots_df[vacant_lots_df['Property Type'] == 'Structure']

# Selecting columns of interest and filtering by 'Property Type' == 'Land' for land dataframe
land_df = vacant_lots_df[vacant_lots_df['Property Type'] == 'Land']

# Filter residential properties
residential_properties = vacant_lots_df[vacant_lots_df['Zoning Category'] == 'Residential']

# Filter commercial properties used Zone Category 3 column for zoned single family residential, commercial and multifamily questions
commercial_properties = vacant_lots_df[vacant_lots_df['Zoning Category'] == 'Commercial']


In [13]:
vacant_lots_df.head()

,Vacant Since,Parcel ID,Address,Property Type,Current Use,Year Built,Total Living Area,Public Owner,City or County Owned,Owner Is Business,Suspected Heir Property,Bank Owned Property,Is Vacant Property,Taxes Delinquent Since,Delinquent Taxes,Is Tax Delinquent,Zip Code,Zoning,Council Districts,Code Liens - Count,Code Liens - Amount,Land Size,Depth,Frontage,Owner Property Count,Total Main Area,Property Age Years,Vacancy Days,Zoning Category 3,Owner is Church,Zoning Category,Owner
0,2023-06-01,"=""00000100051000000""","400 MAIN ST, DALLAS, TX",Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,NaN,NaN,No,Unknown,No,No,Yes,NaN,NaN,No,75202.0,Central Area - CA-1(A)-Central Area,District 2,NaN,NaN,0.918490,NaN,200.0,NaN,NaN,2023.0,90.0,Other,No,Central Area,Private Owner
1,2022-04-01,"=""00000100561000000""","1109 WOOD ST, DALLAS, TX",Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,NaN,NaN,No,Yes,No,No,Yes,NaN,NaN,No,75202.0,Central Area - CA-1(A)-Central Area,District 14,NaN,NaN,0.124653,NaN,50.0,20.0,NaN,2023.0,516.0,Other,No,Central Area,Business
2,2023-06-01,"=""00000101971000000""","1908 ELM ST, DALLAS, TX",Structure,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,NaN,NaN,No,No,No,No,Yes,NaN,NaN,No,75201.0,Other - PD-619,District 14,NaN,NaN,0.122321,NaN,50.0,1.0,NaN,2023.0,90.0,Other,No,Other,Private Owner
3,2022-04-01,"=""00000101995000000""","1913 COMMERCE ST, DALLAS, TX",Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,NaN,City of Dallas,Yes,No,No,No,Yes,NaN,NaN,No,75201.0,Other - PD-619,District 14,NaN,NaN,0.117204,NaN,50.0,2926.0,NaN,2023.0,516.0,Other,No,Other,City of Dallas
4,2022-04-01,"=""00000103702000000""","2523 MAIN ST, DALLAS, TX",Land,C12 - COMMERCIAL - VACANT PLOTTED LOTS/TRACTS ...,0.0,NaN,NaN,No,Yes,No,No,Yes,NaN,NaN,No,75226.0,Other - PD-269,District 2,NaN,NaN,0.126854,NaN,0.0,90.0,NaN,2023.0,516.0,Other,No,Other,Business


In [14]:
vacant_lots_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24901 entries, 0 to 24996
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Vacant Since             24771 non-null  datetime64[ns]
 1   Parcel ID                24901 non-null  object        
 2   Address                  24901 non-null  object        
 3   Property Type            24901 non-null  object        
 4   Current Use              24901 non-null  object        
 5   Year Built               23820 non-null  float64       
 6   Total Living Area        13327 non-null  float64       
 7   Public Owner             1641 non-null   object        
 8   City or County Owned     24901 non-null  object        
 9   Owner Is Business        24901 non-null  object        
 10  Suspected Heir Property  24901 non-null  object        
 11  Bank Owned Property      24901 non-null  object        
 12  Is Vacant Property       24901 non-nu

In [15]:
vacant_lots_df.isnull().sum()

Vacant Since                 130
Parcel ID                      0
Address                        0
Property Type                  0
Current Use                    0
Year Built                  1081
Total Living Area          11574
Public Owner               23260
City or County Owned           0
Owner Is Business              0
Suspected Heir Property        0
Bank Owned Property            0
Is Vacant Property             0
Taxes Delinquent Since     21419
Delinquent Taxes           21419
Is Tax Delinquent              0
Zip Code                       2
Zoning                      4875
Council Districts             13
Code Liens - Count         20782
Code Liens - Amount        20782
Land Size                      0
Depth                      12869
Frontage                     303
Owner Property Count        2473
Total Main Area            11574
Property Age Years          1081
Vacancy Days                 130
Zoning Category 3           4875
Owner is Church                0
Zoning Cat

In [16]:
vacant_lots_df.columns

Index(['Vacant Since', 'Parcel ID', 'Address', 'Property Type', 'Current Use',
       'Year Built', 'Total Living Area', 'Public Owner',
       'City or County Owned', 'Owner Is Business', 'Suspected Heir Property',
       'Bank Owned Property', 'Is Vacant Property', 'Taxes Delinquent Since',
       'Delinquent Taxes', 'Is Tax Delinquent', 'Zip Code', 'Zoning',
       'Council Districts', 'Code Liens - Count', 'Code Liens - Amount',
       'Land Size', 'Depth', 'Frontage', 'Owner Property Count',
       'Total Main Area', 'Property Age Years', 'Vacancy Days',
       'Zoning Category 3', 'Owner is Church', 'Zoning Category', 'Owner'],
      dtype='object')

In [17]:
#Sets results for dict
results = {}

In [18]:
#1.How many vacant lots are in each council district?

#Groups Council distrcits column and 
vacant_lots_counts_by_district  = vacant_lots_df.groupby('Council Districts').size()


In [19]:
#dictonary that starts the result values
results["How many vacant lots are in each council district?"] = {
    'question_number': 1,
    'value': vacant_lots_counts_by_district
}

results


{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64}}

In [20]:
#2 How long have the vacant lots in each council district been vacant? 
vacant_lots_stats = vacant_lots_df.groupby('Council Districts')['Vacancy Days'].agg(['min','median', 'mean', 'max'])

# Rename the columns for clarity
vacant_lots_stats.columns = ['Min Days', 'Median Days', 'Mean Days', 'Max Days']


In [21]:

results["How long have the vacant lots in each council district been vacant?"] = {
    'question_number': 2,
    'value': vacant_lots_stats
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [22]:
# 3 How many are zoned single family residential, commercial and multifamily? <-- rows are council districts; columns are counts within each of the 3 zoning types

# Group by 'Council Districts' and 'Zoning Category', then count occurrences
#zoning_category_counts = vacant_lots_df.groupby(['Council Districts', 'Zoning Category']).size().unstack(fill_value=0)


In [23]:

'''results[" How many are zoned single family residential, commercial and multifamily? <-- rows are council districts; columns are counts within each of the 3 zoning types"] = {
    'question_number': 3,
    'value': zoning_category_counts
}

results'''

'results[" How many are zoned single family residential, commercial and multifamily? <-- rows are council districts; columns are counts within each of the 3 zoning types"] = {\n    \'question_number\': 3,\n    \'value\': zoning_category_counts\n}\n\nresults'

In [24]:
#3 How many vacant lots in each council district are zoned single family residential, commercial and multifamily 

# Group by 'Council Districts' and 'Zoning Category', then count occurrences
zoning_category_counts_3 = vacant_lots_df.groupby(['Council Districts', 'Zoning Category 3']).size().unstack(fill_value=0)


In [25]:

results[" How many are zoned single family residential, commercial and multifamily?"] = {
    'question_number': 3,
    'value': zoning_category_counts_3
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [26]:
# 4. How many vacant structures are in each council district?

# Groups Council districts by structure and vacant property

# Grouping by 'Council Districts' and counting occurrences
vacant_structure_count = structure_df.groupby('Council Districts').size()

In [27]:

results["How many vacant structures are in each council district?"] = {
    'question_number': 4,
    'value': vacant_structure_count
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [28]:
#5 How long have the vacant structures in each council district been vacant? (see question 2)
vacant_structure_count_days = structure_df.groupby('Council Districts')['Vacancy Days'].agg(['min','median', 'mean', 'max'])

# Rename the columns for clarity
vacant_structure_count_days.columns = ['Min Days', 'Median Days', 'Mean Days', 'Max Days']


In [29]:

results["How long have the vacant structures in each council district been vacant? (see question 2)"] = {
    'question_number': 5,
    'value': vacant_structure_count_days
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [30]:

#6 How many vacant structures in each council district are zoned single family residential, commercial and multifamily? (see question 3)

zoning_category_counts_structure = structure_df.groupby(['Council Districts', 'Zoning Category 3']).size().unstack(fill_value=0)


In [31]:

results["How many vacant structures in each council district are zoned single family residential, commercial and multifamily? (see question 3)"] = {
    'question_number': 6,
    'value': zoning_category_counts_structure
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [32]:
#7.How many vacant lots are in each zip code?

# Groups the zip code and if property is vacant and produces the count 
vacant_lots_counts_by_zip = vacant_lots_df.groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_lots_counts_by_zip.index = vacant_lots_counts_by_zip.index.astype(int)

In [33]:

results["How many vacant lots are in each zip code?"] = {
    'question_number': 7,
    'value': vacant_lots_counts_by_zip
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [34]:
#8 How long have the vacant lots in each zip code been vacant?
vacant_zips_count_days = vacant_lots_df.groupby('Zip Code')['Vacancy Days'].agg(['min','median', 'mean', 'max'])

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_zips_count_days.index = vacant_zips_count_days.index.astype(int)

# Rename the columns for clarity
vacant_zips_count_days.columns = ['Min Days', 'Median Days', 'Mean Days', 'Max Days']

In [35]:

results["How long have the vacant lots in each zip code been vacant?"] = {
    'question_number': 8,
    'value': vacant_zips_count_days
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [36]:
#9 How many vacant lots in each zip code are zoned single family residential, commercial and multifamily?


zoning_category_counts_zip = vacant_lots_df.groupby(['Zip Code', 'Zoning Category 3']).size().unstack(fill_value=0)

# Convert index (zip codes) to integers to remove the .0 decimal part
zoning_category_counts_zip.index = zoning_category_counts_zip.index.astype(int)

In [37]:

results["How many vacant lots in each zip code are zoned single family residential, commercial and multifamily?"] = {
    'question_number': 9,
    'value': zoning_category_counts_zip
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [38]:
#10 How many vacant structures are in each zip code?

# Groups the zip code and if property is vacant and produces the count 
vacant_structure_counts_by_zip = structure_df.groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_structure_counts_by_zip.index = vacant_structure_counts_by_zip.index.astype(int)

In [39]:

results["How many vacant structures are in each zip code?"] = {
    'question_number': 10,
    'value': vacant_structure_counts_by_zip
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [40]:
#11 How long have the vacant structures in each zip code been vacant? (see question 8)
vacant_zips_structure_count_days = structure_df.groupby('Zip Code')['Vacancy Days'].agg(['min','median', 'mean', 'max'])

# Convert index (zip codes) to integers to remove the .0 decimal part
vacant_zips_structure_count_days.index = vacant_zips_structure_count_days.index.astype(int)

# Rename the columns for clarity
vacant_zips_structure_count_days.columns = ['Min Days', 'Median Days', 'Mean Days', 'Max Days']

In [41]:

results["How long have the vacant structures in each zip code been vacant? (see question 8)"] = {
    'question_number': 11,
    'value': vacant_zips_structure_count_days
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [42]:
#12 How many vacant structures in each zip code are zoned single family residential, commercial and multifamily? (see question 9)

zoning_category_counts_structure_zip = structure_df.groupby(['Zip Code', 'Zoning Category 3']).size().unstack(fill_value=0)

# Convert index (zip codes) to integers to remove the .0 decimal part
zoning_category_counts_structure_zip.index = zoning_category_counts_structure_zip.index.astype(int)

In [43]:

results["How many vacant structures in each zip code are zoned single family residential, commercial and multifamily? (see question 9)"] = {
    'question_number': 12,
    'value': zoning_category_counts_structure_zip
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [44]:
#13 What is the total amount of back taxes owed on vacant land in each City Council District?

# Grouping and calculating the total amount of back taxes owed on vacant land by City Council District
#Remove vacant prop yes filter
back_taxes_total_by_district = land_df.groupby('Council Districts')['Delinquent Taxes'].sum()

In [45]:

results["What is the total amount of back taxes owed on vacant land in each City Council District?"] = {
    'question_number': 13,
    'value': back_taxes_total_by_district
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [46]:
#14 What is the total amount of back taxes owed on vacant land in each zip code?

# Grouping and calculating the total amount of back taxes owed on vacant land by Zip Code
back_taxes_total_by_zip = land_df.groupby('Zip Code')['Delinquent Taxes'].sum()

# Convert index (zip codes) to integers to remove the .0 decimal part
back_taxes_total_by_zip.index = back_taxes_total_by_zip.index.astype(int)


In [47]:

results["What is the total amount of back taxes owed on vacant land in each zip code?"] = {
    'question_number': 14,
    'value': back_taxes_total_by_zip
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [48]:
#15 What is the total amount of back taxes owed on vacant structures in each City Council District?

# Grouping and calculating the total amount of back taxes owed on vacant land by City Council District
back_taxes_total_by_district_by_structure = structure_df.groupby('Council Districts')['Delinquent Taxes'].sum()


In [49]:

results["What is the total amount of back taxes owed on vacant structures in each City Council District?"] = {
    'question_number': 15,
    'value': back_taxes_total_by_district_by_structure
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [50]:
#16 What is the total amount of back taxes owed on vacant structures in each zip code?

# Grouping and calculating the total amount of back taxes owed on vacant land by Zip Code
back_taxes_total_by_zip_by_structure = structure_df.groupby('Zip Code')['Delinquent Taxes'].sum()

# Convert index (zip codes) to integers to remove the .0 decimal part
back_taxes_total_by_zip_by_structure.index = back_taxes_total_by_zip_by_structure.index.astype(int)


In [51]:

results["What is the total amount of back taxes owed on vacant structures in each zip code?"] = {
    'question_number': 16,
    'value': back_taxes_total_by_zip_by_structure
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [52]:
#17 What is the average age of residential properties in each council district?

avg_age__prop_districts=vacant_lots_df.groupby('Council Districts')['Property Age Years'].mean()


In [53]:

results["What is the average age of residential properties in each council district?"] = {
    'question_number': 17,
    'value': avg_age__prop_districts
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [54]:
#18 What is the average age of residential properties in each zip code?

avg_age__prop_zip=vacant_lots_df.groupby('Zip Code')['Property Age Years'].mean()

#Fills na values with 0
avg_age__prop_zip=avg_age__prop_zip.fillna(0)

#converts avg age to int data type to remve decimals
avg_age__prop_zip.index=avg_age__prop_zip.index.astype(int)


In [55]:

results["What is the average age of residential properties in each zip code?"] = {
    'question_number': 18,
    'value': avg_age__prop_zip
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [56]:
#19 What is the average size of residential properties in each council district?


# Group by council district and calculate the average property size
average_size_by_district = residential_properties.groupby('Council Districts')['Land Size'].mean()

In [57]:

results["What is the average size of residential properties in each council district?"] = {
    'question_number': 19,
    'value': average_size_by_district
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [58]:
#20 What is the average size of residential properties in each zip code?

# Group by council district and calculate the average property size
average_size_by_zip = residential_properties.groupby('Zip Code')['Land Size'].mean()

# Convert index (zip codes) to integers to remove the .0 decimal part
average_size_by_zip.index = average_size_by_zip.index.astype(int)


In [59]:

results["What is the average size of residential properties in each zip code?"] = {
    'question_number': 20,
    'value': average_size_by_zip
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [60]:
#21 What is the average age of commercial properties in each council district?

# Group by council district and calculate the average property size by commerical prop
average_age_by_district_commerical = commercial_properties.groupby('Council Districts')['Property Age Years'].mean()


In [61]:

results["What is the average age of commercial properties in each council district?"] = {
    'question_number': 21,
    'value': average_age_by_district_commerical
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [62]:
#22 What is the average age of commercial properties in each zip code?

average_age_by_zip_commerical = commercial_properties.groupby('Zip Code')['Property Age Years'].mean()

# Convert index (zip codes) to integers to remove the .0 decimal part
average_age_by_zip_commerical.index = average_age_by_zip_commerical.index.astype(int)

In [63]:
results["What is the average age of commercial properties in each zip code?"] = {
    'question_number': 22,
    'value': average_age_by_zip_commerical
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [64]:
#23 What is the total amount of city liens owed on vacant land in each City Council District?

# Grouping and calculating the total amount of city liens owed on vacant land by City Council District
city_liens_total_by_district = land_df.groupby('Council Districts')['Code Liens - Amount'].sum()


In [65]:

results["What is the total amount of city liens owed on vacant land in each City Council District?"] = {
    'question_number': 23,
    'value': city_liens_total_by_district
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [66]:
#24 What is the total amount of city liens owed on vacant land in each zip code?

# Grouping and calculating the total amount of city liens owed on vacant land by City Council District
city_liens_total_by_zip = land_df.groupby('Zip Code')['Code Liens - Amount'].sum()

# Convert index (zip codes) to integers to remove the .0 decimal part
city_liens_total_by_zip.index = city_liens_total_by_zip.index.astype(int)


In [67]:

results["What is the total amount of city liens owed on vacant land in each zip code?"] = {
    'question_number': 24,
    'value': city_liens_total_by_zip
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [68]:
#25 What is the total amount of city liens owed on vacant structures in each City Council District?

# Grouping and calculating the total amount of city liens owed on vacant structures by City Council District
city_liens_total_by_district_by_structure = structure_df.groupby('Council Districts')['Code Liens - Amount'].sum()


In [69]:

results[" What is the total amount of city liens owed on vacant structures in each City Council District?"] = {
    'question_number': 25,
    'value': city_liens_total_by_district_by_structure
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [70]:
#26 How many properties are in the name of heirship in the City of Dallas?

#Use suspected heir prop col and combine with public owner filter
# Filter the DataFrame for properties in the City of Dallas with 'Suspected Heir Property' == 'Yes'
#Would get zero combined with City of Dallas filter
dallas_heirship_properties = (vacant_lots_df['Suspected Heir Property'] == 'Yes').sum()



In [71]:

results["How many properties are in the name of heirship in the City of Dallas?"] = {
    'question_number': 26,
    'value': dallas_heirship_properties
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [72]:
#27 How many properties in the name of heirship in the City of Dallas are vacant in each zip code?

#apply Suspected heir to Yes and group by zip code
dallas_heirship_zip = vacant_lots_df[vacant_lots_df['Suspected Heir Property'] == 'Yes'].groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
dallas_heirship_zip.index = dallas_heirship_zip.index.astype(int)


In [73]:

results["How many properties in the name of heirship in the City of Dallas are vacant in each zip code?"] = {
    'question_number': 27,
    'value': dallas_heirship_zip
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [74]:
#28 How many properties in the name of heirship in the City of Dallas are vacant in each council district?

#apply Suspected heir to Yes and group by council distrcit
dallas_heirship_districts = vacant_lots_df[vacant_lots_df['Suspected Heir Property'] == 'Yes'].groupby('Council Districts').size()


In [75]:

results["How many properties in the name of heirship in the City of Dallas are vacant in each council district?"] = {
    'question_number': 28,
    'value': dallas_heirship_districts
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [76]:
#29 How many private owners have five or more vacant properties in the City of Dallas?
owner_vacant_property_counts = vacant_lots_df[
    (vacant_lots_df['Owner'] == 'Private Owner') &
    (vacant_lots_df['Owner Property Count'] >= 5)
]

owner_vacant_property_counts_view = owner_vacant_property_counts.shape[0]


In [77]:

results["How many private owners have five or more vacant properties in the City of Dallas?"] = {
    'question_number': 29,
    'value': owner_vacant_property_counts_view
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [78]:
#30 How many private owners with five or more vacant properties in the City of Dallas are in each zip code?

private_5_zip = owner_vacant_property_counts.groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
private_5_zip.index = private_5_zip.index.astype(int)


In [79]:

results["How many private owners with five or more vacant properties in the City of Dallas are in each zip code?"] = {
    'question_number': 30,
    'value': private_5_zip
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [80]:
#31 How many private owners with five or more vacant properties in the City of Dallas are in each council district?

private_5_district=owner_vacant_property_counts.groupby('Council Districts').size()

In [81]:

results[" How many private owners with five or more vacant properties in the City of Dallas are in each council district?"] = {
    'question_number': 31,
    'value': private_5_district
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [82]:
#32 How many churches own vacant properties in the City of Dallas?

church_owners = vacant_lots_df[vacant_lots_df['Owner is Church'] == 'Yes']

num_church_properties = church_owners.shape[0]



In [83]:

results["How many churches own vacant properties in the City of Dallas?"] = {
    'question_number': 32,
    'value': num_church_properties
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [84]:
#33 How many churches owned vacant properties in the City of Dallas are in each zip code?

church_prop_zip = vacant_lots_df[vacant_lots_df['Owner is Church'] == 'Yes'].groupby('Zip Code').size()

# Convert index (zip codes) to integers to remove the .0 decimal part
church_prop_zip.index = church_prop_zip.index.astype(int)


In [85]:

results["How many churches owned vacant properties in the City of Dallas are in each zip code?"] = {
    'question_number': 33,
    'value': church_prop_zip
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric

In [86]:
#34 How many are in each council district

church_prop_district = vacant_lots_df[vacant_lots_df['Owner is Church'] == 'Yes'].groupby('Council Districts').size()


In [87]:

results["How many are in each council district"] = {
    'question_number': 34,
    'value': church_prop_district
}

results

{'How many vacant lots are in each council district?': {'question_number': 1,
  'value': Council Districts
  District 1     1233
  District 10     498
  District 11     555
  District 12     216
  District 13     983
  District 14    1182
  District 2     2340
  District 3     1437
  District 4     3552
  District 5     1541
  District 6     3857
  District 7     4047
  District 8     2829
  District 9      618
  dtype: int64},
 'How long have the vacant lots in each council district been vacant?': {'question_number': 2,
  'value':                    Min Days  Median Days   Mean Days  Max Days
  Council Districts                                             
  District 1             90.0        516.0  479.136919     516.0
  District 10            90.0        516.0  446.271084     516.0
  District 11            90.0        516.0  455.556364     516.0
  District 12            90.0        516.0  442.957944     516.0
  District 13            90.0        516.0  451.408350     516.0
  Distric